In [1]:
import pandas as pd
import numpy as np
import numexpr as ne
from memory_profiler import profile

| Araç              | Ne İşe Yarar?                     | Ne Zaman Kullanılır?                  |
| ----------------- | --------------------------------- | ------------------------------------- |
| `query`           | DataFrame filtreleme (daha hızlı) | Pandas veri çekerken                  |
| `eval`            | Sütunlar arasında hesaplama vs    | Hafif işlemler, sütun üretme          |
| `numexpr`         | Büyük verilerde hızlı matematik   | Büyük NumPy dizileriyle çalışırken    |
| `%timeit`         | Kod süresi ölçümü                 | Kısa kod bloklarını test ederken      |
| `%prun`           | Satır satır süre ölçer            | Fonksiyon detaylı analizinde          |
| `memory_profiler` | RAM tüketimi ölçümü               | Bellek kaçıran kodları tespit ederken |


# random dataframe oluşturma

In [2]:
df = pd.DataFrame({
    'x': np.random.rand(10_000_000),
    'y': np.random.rand(10_000_000),
    'z': np.random.randint(0, 100, size=10_000_000)
})

In [3]:
df

,x,y,z
0,0.420738,0.641762,54
1,0.960884,0.246208,13
2,0.945917,0.193057,78
3,0.330546,0.021709,58
4,0.283764,0.314578,88
...,...,...,...
9999995,0.214084,0.837924,19
9999996,0.575523,0.051352,54
9999997,0.899034,0.834395,42
9999998,0.536390,0.727793,58


## klasik yöntem ile toplama ve filtreleme

In [5]:
filtered = df[(df['y'] < 0.3) & (df['x'] > 0.5)]
df['sum1'] = df['x'] + df['y']

## pd.DataFrame.query() ile filtreleme

In [ ]:
filtered_q = df.query("x > 0.5 and y < 0.3")


## pd.DataFrame.eval() ile toplama

In [7]:
df.eval("sum2 = x + y", inplace=True)

## numexpr ile daha hızlı toplama (C based lib)

In [8]:
import numexpr as ne
df['sum3'] = ne.evaluate("x + y", local_dict={'x': df['x'].values, 'y': df['y'].values})

# Zaman karşılaştırmaları

## kod süresi ölçümü

In [9]:
%timeit df['x'] + df['y']
%timeit df.eval("x + y")
%timeit ne.evaluate("x + y", local_dict={'x': df['x'].values, 'y': df['y'].values})


17.5 ms ± 469 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
58.1 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
19.1 ms ± 850 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## satır satır süre ölçümü

In [10]:
def klasik_toplama():
    df['toplam'] = df['x'] + df['y']

%prun klasik_toplama()

         989 function calls (971 primitive calls) in 0.028 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.009    0.009    0.009    0.009 {method 'copy' of 'numpy.ndarray' objects}
        1    0.007    0.007    0.007    0.007 necompiler.py:979(re_evaluate)
        2    0.006    0.003    0.006    0.003 {method '__exit__' of 'sqlite3.Connection' objects}
        1    0.002    0.002    0.009    0.009 necompiler.py:895(evaluate)
        3    0.000    0.000    0.002    0.001 base_events.py:1922(_run_once)
      3/1    0.000    0.000    0.001    0.001 events.py:86(_run)
       10    0.000    0.000    0.000    0.000 socket.py:626(send)
  219/214    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 series.py:389(__init__)
        3    0.000    0.000    0.00

# `memory_profiler` ile RAM kullanımı

## `.py` uzantılı dosyalar için --> `profile`

In [12]:
from memory_profiler import profile

@profile
def hesapla():
    df['toplam'] = df['x'] + df['y']

hesapla()

ERROR: Could not find file C:\Users\Umut\AppData\Local\Temp\ipykernel_6556\2336999538.py


## `.ipynb` için alternatif kullanım --> `memit`

In [13]:
%load_ext memory_profiler

def hesapla():
    df = pd.DataFrame({
        'x': np.random.rand(10_000_000),
        'y': np.random.rand(10_000_000),
    })
    df['toplam'] = df['x'] + df['y']
    return df

%memit hesapla()

peak memory: 931.15 MiB, increment: 136.36 MiB
